# Data Cleaning

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

import os

files = os.listdir("data")
for file in files:
    print(file)

Product Reports 01 January 2023 - 31 December 2023.csv
Product Reports 01 January 2024 - 29 February 2024.csv
Product Reports 01 January 2021 - 31 December 2021.csv
Product Reports 01 January 2022 - 31 December 2022.csv


Data Loading

In [2]:
import re
from datetime import datetime


dfs = []
for file in files:
    # match = re.search(r'\d{2} (.*?) \d{4}', file)
    year = re.search(r'\d{4}', file).group()
    # date = datetime.strptime(match.group(), r"%d %M %Y").date()
    # print(match.group())
    df = pd.read_csv(f"data/{file}")
    df["year"] = int(year)
    dfs.append(df)

df = pd.concat(dfs)

In [3]:
df.head()

,Menu Item Name,Menu Category,QTY sold,Sales exc Tax.,Discounts,Sales inc Tax.(a),Cost inc Tax(b),Gross Profit(a)-(b),Markup Percentage %(a-b)/b * 100%,year
0,box(-),NaN,1550,£ 5889.10,£ 24.26,£ 5889.10,£ 0.00,£ 5889.10,0 %,2023
1,75. Rib Eye(Standard),BBQ (Korean) 75 - 84,598,£ 8491.85,£ 55.76,£ 8916.44,£ 0.00,£ 8916.44,0 %,2023
2,65D. Egg(Standard),Extras,2,£ 0.95,£ 0.00,£ 1.00,£ 0.00,£ 1.00,0 %,2023
3,76. Dwaeji galbi ribs(default),BBQ (Korean) 75 - 84,220,£ 2499.38,£ 10.65,£ 2624.35,£ 0.00,£ 2624.35,0 %,2023
4,77. beef Bulgogi(Standard),BBQ (Korean) 75 - 84,867,£ 9773.04,£ 121.61,£ 10261.69,£ 0.00,£ 10261.69,0 %,2023


In [4]:
df.shape

(1400, 10)

Renaming columns for ease of manipulation

In [5]:
df.columns

Index(['Menu Item Name', 'Menu Category', 'QTY sold', 'Sales exc Tax.',
       'Discounts', 'Sales inc Tax.(a)', 'Cost inc Tax(b)',
       'Gross Profit(a)-(b)', 'Markup Percentage %(a-b)/b * 100%', 'year'],
      dtype='object')

In [6]:
df.columns = ["name", "category", "sold", "sales_inc_tax", "discount", "sales_exc_tax", "cost_inc_tax", "gross_profit", "markup", "year"]

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1400 entries, 0 to 349
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   name           1400 non-null   object
 1   category       1392 non-null   object
 2   sold           1400 non-null   int64 
 3   sales_inc_tax  1400 non-null   object
 4   discount       1400 non-null   object
 5   sales_exc_tax  1400 non-null   object
 6   cost_inc_tax   1400 non-null   object
 7   gross_profit   1400 non-null   object
 8   markup         1400 non-null   object
 9   year           1400 non-null   int64 
dtypes: int64(2), object(8)
memory usage: 120.3+ KB


Columns that are supposed to be `float` are being parsed as `object`. This is due to the values having:
- the currency sign (which in this case is `£`)
- the percentage sign `%`

In [8]:
currency_columns = ["sales_inc_tax", "discount", "sales_exc_tax", "cost_inc_tax", "gross_profit"]
percentage_columns = ["markup"]

for col in currency_columns:
    df[col] = df[col].str.replace("£", "").str.strip().astype(float)

for col in percentage_columns:
    df[col] = df[col].str.replace("%", "").str.strip().astype(float)

df.head()

,name,category,sold,sales_inc_tax,discount,sales_exc_tax,cost_inc_tax,gross_profit,markup,year
0,box(-),NaN,1550,5889.10,24.26,5889.10,0.0,5889.10,0.0,2023
1,75. Rib Eye(Standard),BBQ (Korean) 75 - 84,598,8491.85,55.76,8916.44,0.0,8916.44,0.0,2023
2,65D. Egg(Standard),Extras,2,0.95,0.00,1.00,0.0,1.00,0.0,2023
3,76. Dwaeji galbi ribs(default),BBQ (Korean) 75 - 84,220,2499.38,10.65,2624.35,0.0,2624.35,0.0,2023
4,77. beef Bulgogi(Standard),BBQ (Korean) 75 - 84,867,9773.04,121.61,10261.69,0.0,10261.69,0.0,2023


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1400 entries, 0 to 349
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           1400 non-null   object 
 1   category       1392 non-null   object 
 2   sold           1400 non-null   int64  
 3   sales_inc_tax  1400 non-null   float64
 4   discount       1400 non-null   float64
 5   sales_exc_tax  1400 non-null   float64
 6   cost_inc_tax   1400 non-null   float64
 7   gross_profit   1400 non-null   float64
 8   markup         1400 non-null   float64
 9   year           1400 non-null   int64  
dtypes: float64(6), int64(2), object(2)
memory usage: 120.3+ KB


As seen above the column types are fixed.

In [10]:
df.describe()

,sold,sales_inc_tax,discount,sales_exc_tax,cost_inc_tax,gross_profit,markup,year
count,1400.000000,1400.000000,1400.000000,1400.000000,1400.0,1400.000000,1400.0,1400.000000
mean,417.558571,2805.504229,37.775300,2935.522671,0.0,2935.522671,0.0,2022.490000
std,4374.167324,29254.448751,409.809853,30607.512298,0.0,30607.512298,0.0,1.083986
min,1.000000,0.000000,-50.000000,0.000000,0.0,0.000000,0.0,2021.000000
25%,21.750000,98.930000,0.000000,103.500000,0.0,103.500000,0.0,2022.000000
50%,80.000000,489.670000,3.370000,513.150000,0.0,513.150000,0.0,2023.000000
75%,236.250000,1572.787500,20.635000,1647.405000,0.0,1647.405000,0.0,2023.000000
max,105672.000000,698744.260000,11407.760000,730360.670000,0.0,730360.670000,0.0,2024.000000


## Empty and duplicate data
Checking for empty values and duplicates

In [11]:
df.duplicated().sum()

0

In [12]:
df.isna().sum()

name             0
category         8
sold             0
sales_inc_tax    0
discount         0
sales_exc_tax    0
cost_inc_tax     0
gross_profit     0
markup           0
year             0
dtype: int64

In [13]:
df[df.category.isna()]

,name,category,sold,sales_inc_tax,discount,sales_exc_tax,cost_inc_tax,gross_profit,markup,year
0,box(-),NaN,1550,5889.10,24.26,5889.10,0.0,5889.10,0.0,2023
402,Total Sales,NaN,100916,693532.78,6095.22,726228.89,0.0,726228.89,0.0,2023
0,cocumber kimchi(-),NaN,196,1629.97,5.71,1629.97,0.0,1629.97,0.0,2024
309,Total Sales,NaN,13778,106910.06,841.21,111991.12,0.0,111991.12,0.0,2024
0,additional charge for sashimi(-),NaN,38,169.93,4.27,169.93,0.0,169.93,0.0,2021
336,Total Sales,NaN,71925,464665.86,11407.76,486285.19,0.0,486285.19,0.0,2021
0,soju(-),NaN,1188,6026.77,60.47,6026.77,0.0,6026.77,0.0,2022
349,Total Sales,NaN,105672,698744.26,8098.52,730360.67,0.0,730360.67,0.0,2022


As seen above, there are some legitimate products with no category. The other ones are `Total Sales` just noise.

I will remove these as they are not needed.

In [14]:
df = df[~(df.name == "Total Sales")]

In [15]:
df.isna().sum()

name             0
category         4
sold             0
sales_inc_tax    0
discount         0
sales_exc_tax    0
cost_inc_tax     0
gross_profit     0
markup           0
year             0
dtype: int64

The remaining products with no category are custom sales that are not at the menu at the time.
Thus I will replace it with `custom` category. 

In [16]:
df  = df.fillna("custom")
df.head()

,name,category,sold,sales_inc_tax,discount,sales_exc_tax,cost_inc_tax,gross_profit,markup,year
0,box(-),custom,1550,5889.10,24.26,5889.10,0.0,5889.10,0.0,2023
1,75. Rib Eye(Standard),BBQ (Korean) 75 - 84,598,8491.85,55.76,8916.44,0.0,8916.44,0.0,2023
2,65D. Egg(Standard),Extras,2,0.95,0.00,1.00,0.0,1.00,0.0,2023
3,76. Dwaeji galbi ribs(default),BBQ (Korean) 75 - 84,220,2499.38,10.65,2624.35,0.0,2624.35,0.0,2023
4,77. beef Bulgogi(Standard),BBQ (Korean) 75 - 84,867,9773.04,121.61,10261.69,0.0,10261.69,0.0,2023


In [17]:
# df.groupby(["year", "name"]).nlargest(2, ["sold"])
df.groupby(['year', "name"]).apply(lambda x: x.nlargest(10, ['sold']))

name  \
year name                                                                             
2021 01 Sette Bello Prosecco(20CL)        19          01 Sette Bello Prosecco(20CL)   
     01 Sette Bello Prosecco(Bottle)      20        01 Sette Bello Prosecco(Bottle)   
     01. Spicy Squid(Standard)            229             01. Spicy Squid(Standard)   
     02 Da Luca Rosato Spumante(Standard) 21   02 Da Luca Rosato Spumante(Standard)   
     02. Prawn Tempura(Standard)          230           02. Prawn Tempura(Standard)   
...                                                                             ...   
2024 Veg Sushi Platter 20pc(-)            218             Veg Sushi Platter 20pc(-)   
     Whiskey(Double)                      204                       Whiskey(Double)   
     Whiskey(Single)                      139                       Whiskey(Single)   
     Whisky Smash                         294                          Whisky Smash   
     cocumber kimchi(-)                   0                      cocumber kimchi(-)   

                                                        category  sold  \
year name                                                                
2021 01 Sette Bello Prosecco(20CL)        19           Champagne    43   
     01 Sette Bello Prosecco(Bottle)      20           Champagne    73   
     01. Spicy Squid(Standard)            229   Traditional 1-15   829   
     02 Da Luca Rosato Spumante(Standard) 21           Champagne    23   
     02. Prawn Tempura(Standard)          230   Traditional 1-15   939   
...                                                          ...   ...   
2024 Veg Sushi Platter 20pc(-)            218     Sushi Platters     5   
     Whiskey(Double)                      204    Spirits & Shots     1   
     Whiskey(Single)                      139    Spirits & Shots     3   
     Whisky Smash                         294  Classic Cocktails    11   
     cocumber kimchi(-)                   0               custom   196   

                                               sales_inc_tax  discount  \
year name                                                                
2021 01 Sette Bello Prosecco(20CL)        19          202.07      2.83   
     01 Sette Bello Prosecco(Bottle)      20         1524.22      5.57   
     01. Spicy Squid(Standard)            229        5722.22    209.17   
     02 Da Luca Rosato Spumante(Standard) 21          409.86      6.65   
     02. Prawn Tempura(Standard)          230        6539.04    176.51   
...                                                      ...       ...   
2024 Veg Sushi Platter 20pc(-)            218         119.05      0.00   
     Whiskey(Double)                      204           4.29      0.00   
     Whiskey(Single)                      139           7.14      0.00   
     Whisky Smash                         294          79.24      4.80   
     cocumber kimchi(-)                   0          1629.97      5.71   

                                               sales_exc_tax  cost_inc_tax  \
year name                                                                    
2021 01 Sette Bello Prosecco(20CL)        19          212.17           0.0   
     01 Sette Bello Prosecco(Bottle)      20         1600.43           0.0   
     01. Spicy Squid(Standard)            229        6008.33           0.0   
     02 Da Luca Rosato Spumante(Standard) 21          430.35           0.0   
     02. Prawn Tempura(Standard)          230        6865.99           0.0   
...                                                      ...           ...   
2024 Veg Sushi Platter 20pc(-)            218         125.00           0.0   
     Whiskey(Double)                      204           4.50           0.0   
     Whiskey(Single)                      139           7.50           0.0   
     Whisky Smash                         294          83.20           0.0   
     cocumber kimchi(-)                   0          1629.97           0.0   

      

In [18]:
# df.groupby(['year', "name"]).apply(lambda x: x.nlargest(10, ['sold'])).reset_index(drop=True)
df.nlargest(10, ["sold"])

,name,category,sold,sales_inc_tax,discount,sales_exc_tax,cost_inc_tax,gross_profit,markup,year
42,Asahi(Pint),Draught Beer,5026,24832.01,61.59,26073.61,0.0,26073.61,0.0,2022
42,Asahi(Pint),Draught Beer,4733,23397.70,44.01,24567.59,0.0,24567.59,0.0,2023
41,Asahi(Pint),Draught Beer,3954,19442.52,146.15,20414.65,0.0,20414.65,0.0,2021
151,Coke(Standard),Soft Drinks,3852,9912.78,18.98,10408.42,0.0,10408.42,0.0,2023
158,Coke(Standard),Soft Drinks,3494,8931.53,55.69,9378.11,0.0,9378.11,0.0,2022
152,Diet Coke(Standard),Soft Drinks,3327,8556.79,17.27,8984.63,0.0,8984.63,0.0,2023
159,Diet Coke(Standard),Soft Drinks,3304,8454.87,43.19,8877.61,0.0,8877.61,0.0,2022
301,Cocktail(-),Cocktails,3013,20119.23,971.77,20119.23,0.0,20119.23,0.0,2021
146,Chilli Fries(Standard),Sides,2537,8409.01,50.04,8829.46,0.0,8829.46,0.0,2023
307,Cocktail(-),Cocktails,2517,17540.88,78.12,17540.88,0.0,17540.88,0.0,2022


In [19]:
df.sort_values(["year", "sold"], ascending=False).groupby(["year", "name"]).head(2)

,name,category,sold,sales_inc_tax,discount,sales_exc_tax,cost_inc_tax,gross_profit,markup,year
34,Asahi(Pint),Draught Beer,676,3350.78,-3.12,3518.32,0.0,3518.32,0.0,2024
121,Coke(Standard),Soft Drinks,475,1221.17,0.27,1282.23,0.0,1282.23,0.0,2024
122,Diet Coke(Standard),Soft Drinks,422,1083.13,2.11,1137.29,0.0,1137.29,0.0,2024
28,25.Chicken Katsu Curry(Standard),Japan rice dis 25-32,387,5071.51,54.21,5325.09,0.0,5325.09,0.0,2024
116,Chilli Fries(Standard),Sides Extras,364,1207.12,6.52,1267.48,0.0,1267.48,0.0,2024
...,...,...,...,...,...,...,...,...,...,...
188,Hendricks(Single),Spirits,1,3.62,0.00,3.80,0.0,3.80,0.0,2021
201,Nigiri (3pc)(Tamago),Sushi,1,3.05,0.80,3.20,0.0,3.20,0.0,2021
277,Amaretto(Double),Spirits,1,3.96,0.74,4.16,0.0,4.16,0.0,2021
282,Brandy(Double),Spirits,1,5.52,0.00,5.80,0.0,5.80,0.0,2021


## Data Standardisation
- name and categories must be lowercase
- some names are different but refer to same product: combine these
- some categories are different but refer to same category: combine these
- remove unnecessary noise from the names and categories e.g. product number

In [20]:
df.name = df.name.str.lower()
df.category = df.category.str.lower()


In [21]:
df.category.unique()

array(['custom', 'bbq (korean) 75 - 84', 'extras', 'bottled beer & cider',
       'buns 24-28', 'champagne', 'cocktails', 'dessert', 'donburi 29-39',
       'draught beer', 'korean nood 62-65', 'korean set menu',
       'korean sides 49-52', 'korean soft drinks', 'korean soju',
       'korean starter 53-61', 'noodles 47-48', 'platters 45-46',
       'red wine', 'rice dishes 70-74', 'robata grill 15-23', 'rose wine',
       'sides', 'soft drinks', 'restaurant specials', 'soup 66-69',
       'special mains 39-46', 'spirits', 'sushi', 'sushi platters',
       'sushi rolls (4pc)', 'tea / coffee', 'traditional 1-14',
       'white wine', 'new items', 'spirits & shots', 'vegan',
       'lunch bento', 'seared salmon nigiri', 'restaurant special',
       'sushi rolls 4pc/8pc', 'mocktails', 'asian cocktails',
       'classic cocktails', 'grill bbq 52-60', 'buns 33-38',
       'japan rice dis 25-32', 'korean dishes 41-51', 'sides extras',
       'korean side 14-24', 'ramen & noodle 39-40', 'bbq 

In [22]:
df.name.unique()

array(['box(-)', '75. rib eye(standard)', '65d. egg(standard)',
       '76. dwaeji galbi ribs(default)', '77. beef bulgogi(standard)',
       '78. pork bulgogi(default)', '79. chicken bulgogi(default)',
       '80. pork belly(standard)', '81. king prawns (5pc)(standard)',
       '82. ox tongue(standard)', '83. chargrilled squid(standard)',
       '84. lettuce, garlic & chilli(standard)', 'becks blue(standard)',
       'corona(standard)', 'kirin ichi ban(standard)',
       'koppaberg(standard)', 'tiger(standard)', 'tsing tao(standard)',
       '25. chicken teriyaki(standard)', '26. beef teriyaki(standard)',
       '01 sette bello prosecco(20cl)', '01 sette bello prosecco(bottle)',
       '02 da luca rosato spumante(standard)',
       '03 da luca prosecco(standard)',
       '04 taittinger brut reserve nv(standard)', 'asia daisy(standard)',
       'japanese slipper(standard)', 'korean yoghurt soju(standard)',
       'matcha mojito(standard)', 'plum sour(standard)',
       'cheesecake(stan

In [23]:
df[df.category.str.lower().str.contains("cocktail")]

,name,category,sold,sales_inc_tax,discount,sales_exc_tax,cost_inc_tax,gross_profit,markup,year
25,asia daisy(standard),cocktails,469,3143.14,21.70,3300.30,0.0,3300.30,0.0,2023
26,japanese slipper(standard),cocktails,121,816.38,0.80,857.20,0.0,857.20,0.0,2023
27,korean yoghurt soju(standard),cocktails,120,811.19,3.25,851.75,0.0,851.75,0.0,2023
28,matcha mojito(standard),cocktails,108,724.10,0.70,760.30,0.0,760.30,0.0,2023
29,plum sour(standard),cocktails,115,777.14,0.00,816.00,0.0,816.00,0.0,2023
266,mocktail(-),cocktails,1109,4219.56,23.46,4430.54,0.0,4430.54,0.0,2023
296,cocktail(-),cocktails,1499,10462.16,30.84,10462.16,0.0,10462.16,0.0,2023
317,cocktaill(-),cocktails,164,1309.60,2.40,1309.60,0.0,1309.60,0.0,2023
381,for cherry’s “sake”,asian cocktails,13,111.43,0.00,117.00,0.0,117.00,0.0,2023
382,mojito(passion fruit),asian cocktails,14,106.63,0.04,111.96,0.0,111.96,0.0,2023
